In [112]:
import pandas as pd 
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 
class CF(object):
    """Khởi tạo class CF
        Dữ liệu đầu vào của hàm khởi tạo class CF là ma trận Utility Y_data được lưu dưới dạng một ma trận với 3 cột, 
        k là số lượng các điểm lân cận được sử dụng để dự đoán kết quả. dist_func là hàm đó similarity giữa hai vectors, mặc định là cosine_similarity được lấy từ sklearn.metrics.pairwise. Bạn đọc cũng có thể thử với các giá trị k và hàm dist_func khác nhau. 
        Biến uuCF thể hiện việc đang sử dụng User-user CF (1) hay Item-item CF(0).
    """
    def __init__(self, Y_data, k, dist_func = cosine_similarity, uuCF = 1):
        self.uuCF = uuCF # user-user (1) or item-item (0) CF
        self.Y_data = Y_data if uuCF else Y_data[:, [1, 0, 2]]
        self.k = k
        self.dist_func = dist_func
        self.Ybar_data = None
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1 
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1
    
    def add(self, new_data):
        """
        Khi có dữ liệu mới, cập nhận Utility matrix bằng cách thêm các hàng này vào cuối Utility Matrix.
        Để cho đơn giản, giả sử rằng không có users hay items mới, cũng không có ratings nào bị thay đổi.
        """
        self.Y_data = np.concatenate((self.Y_data, new_data), axis = 0)
    #Tính toán normalized utility matrix và Similarity matrix
    def normalize_Y(self):
        users = self.Y_data[:, 0] # all users - first col of the Y_data
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(self.n_users):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, 1] 
            # and the corresponding ratings 
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Ybar_data[ids, 2] = ratings - self.mu[n]

        ################################################
        # form the rating matrix as a sparse matrix. Sparsity is important 
        # for both memory and computing efficiency. For example, if #user = 1M, 
        # #item = 100k, then shape of the rating matrix would be (100k, 1M), 
        # you may not have enough memory to store this. Then, instead, we store 
        # nonzeros only, and, of course, their locations.
        self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
            (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_items, self.n_users))
        self.Ybar = self.Ybar.tocsr()

    def similarity(self):
        eps = 1e-6
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
    
    #Thực hiện lại 2 hàm phía trên khi có thêm dữ liệu.
    def refresh(self):
        """
        Normalize data and calculate similarity matrix again (after
        some few ratings added)
        """
        self.normalize_Y()
        self.similarity() 
        
    def fit(self):
        self.refresh()
        
    #Dự đoán kết quả:
    '''
        Hàm __pred là hàm dự đoán rating mà user u cho item i cho trường hợp User-user CF. 
        Vì trong trường hợp Item-item CF, chúng ta cần hiểu ngược lại nên hàm pred sẽ thực hiện đổi vị trí hai biến của __pred. 
        Để cho API được đơn giản, tôi cho __pred là một phương thức private, chỉ được gọi trong class CF; 
        pred là một phương thức public,
        thứ tự của biến đầu vào luôn là (user, item), bất kể phương pháp sử dụng là User-user CF hay Item-item CF.
    '''
    def __pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        # Step 1: find all users who rated i
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        # Step 2: 
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        # Step 3: find similarity btw the current user and others 
        # who already rated i
        sim = self.S[u, users_rated_i]
        # Step 4: find the k most similarity users
        a = np.argsort(sim)[-self.k:] 
        # and the corresponding similarity levels
        nearest_s = sim[a]
        # How did each of 'near' users rated item i
        r = self.Ybar[i, users_rated_i[a]]
        if normalized:
            # add a small number, for instance, 1e-8, to avoid dividing by 0
            return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8)

        return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8) + self.mu[u]
    
    def pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        if self.uuCF: return self.__pred(u, i, normalized)
        return self.__pred(i, u, normalized)
            
    '''
    Tìm tất cả các items nên được gợi ý cho user u trong trường hợp User-user CF,
    hoặc tìm tất cả các users có khả năng thích item u trong trường hợp Item-item CF
    '''
    def recommend(self, u):
        """
        Determine all items should be recommended for user u.
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which 
        have not been rated by u yet. 
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()              
        recommended_items = []
        for i in xrange(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0: 
                    recommended_items.append(i)
        
        return recommended_items 
    
    def recommend2(self, u):
        """
        Determine all items should be recommended for user u.
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which 
        have not been rated by u yet. 
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()              
        recommended_items = []
    
        for i in xrange(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0: 
                    recommended_items.append(i)
        
        return recommended_items 

    def print_recommendation(self):
        """
        print all items which should be recommended for each user 
        """
        print('Recommendation: ')
        for u in range(self.n_users):
            recommended_items = self.recommend(u)
            if self.uuCF:
                print(' Recommend item(s):', recommended_items, 'for user', u)
            else: 
                print(' Recommend item', u, 'for user(s) : ', recommended_items)

Chúng ta cùng quay lại làm với cơ sở dữ liệu MoiveLens 100k như trong Content-based Recommendation Systems. Nhắc lại rằng kết quả của phương pháp này có trung bình lỗi là 1.2 sao với mỗi rating.

Chúng ta cùng xem kết quả với User-user CF và Item-item CF.

Trước hết, ta cần load dữ liệu.

In [113]:
import math
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base
rate_test = ratings_test

# indices start from 0
rate_train.iloc[:, :2] -= 1
rate_test.iloc[:, :2] -= 1


- kết quả với User-user cf

In [116]:
rs = CF(rate_train.values, k = 50, uuCF = 1)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test.iloc[n, 0], rate_test.iloc[n, 1], normalized = 0)
    SE += (pred - rate_test.iloc[n, 2])**2 

RMSE = math.sqrt(SE/n_tests)
print('User-user CF, RMSE =', RMSE)

User-user CF, RMSE = 0.9959828811717598


- Kết quả với Item-item cf

In [117]:
rs = CF(rate_train.values, k = 55, uuCF = 0)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test.iloc[n, 0], rate_test.iloc[n, 1], normalized = 0)
    SE += (pred - rate_test.iloc[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print('Item-item CF, RMSE =', RMSE)

Item-item CF, RMSE = 0.990340104487686
